In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)

In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [33]:
X = torch.load('data_processed/1face_X_part0.pt', map_location = device)

In [34]:
Y = torch.load('data_processed/1face_Y_part0.pt', map_location = device)

In [35]:
X = X[0:1000]
Y = Y[0:1000]

In [39]:
X[0]

tensor([[ 0.0542, -0.0126,  0.0381,  ...,  0.0216,  0.0413, -0.0247],
        [ 0.0563, -0.0216,  0.0402,  ...,  0.0129,  0.0312, -0.0161],
        [ 0.0532, -0.0239,  0.0438,  ...,  0.0130,  0.0227, -0.0050],
        ...,
        [ 0.0626, -0.0222,  0.0392,  ...,  0.0041,  0.0281, -0.0107],
        [ 0.0595, -0.0230,  0.0349,  ...,  0.0084,  0.0344, -0.0134],
        [ 0.0621, -0.0222,  0.0431,  ...,  0.0090,  0.0265, -0.0152]])

In [40]:
X = torch.stack(X)

In [41]:
X.shape

torch.Size([1000, 30, 512])

In [42]:
import torch.nn as nn

class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.9):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.9)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.float()
        
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [43]:
input_size = 512
output_size = 1
hidden_dim = 30
n_layers = 2

model = DFDCNet(input_size, output_size, hidden_dim, n_layers)
model.to(device)
print(model)

DFDCNet(
  (lstm): LSTM(512, 30, num_layers=2, batch_first=True, dropout=0.9)
  (dropout): Dropout(p=0.9, inplace=False)
  (fc): Linear(in_features=30, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [44]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [45]:
from torch.utils.data import TensorDataset, DataLoader
train_data = TensorDataset(X, torch.from_numpy(np.array(Y)))
batch_size = 100

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

In [48]:
epochs = 2
counter = 0
print_every = 10
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './model_1face.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

Epoch: 1/2... Step: 10... Loss: 0.395854... Val Loss: 0.291519
Validation loss decreased (inf --> 0.291519).  Saving model ...
Epoch: 2/2... Step: 20... Loss: 0.325059... Val Loss: 0.249807
Validation loss decreased (0.291519 --> 0.249807).  Saving model ...


# Reference
* https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb